In [1]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
import d6tflow
import luigi

Loading postgres module without psycopg2 installed. Will crash at runtime if postgres functionality is used.
Loading S3 module without the python package boto3. Will crash at runtime if S3 functionality is used.


Welcome to d6tflow!


In [ ]:
df_rating = pd.read_csv("~/Downloads/Book reviews/BX-Book-Ratings.csv", sep=";", quotechar='"', engine='python', error_bad_lines=False,  encoding='ISO-8859–1')
df_book = pd.read_csv("~/Downloads/Book reviews/BX-Books.csv", sep=";", quotechar='"', engine='python', error_bad_lines=False,  encoding='ISO-8859–1')
df_user = pd.read_csv("~/Downloads/Book reviews/BX-Users.csv", sep=";", quotechar='"', engine='python', error_bad_lines=False,  encoding='ISO-8859–1')

Skipping line 21689: ';' expected after '"'
Skipping line 45329: ';' expected after '"'
Skipping line 45330: ';' expected after '"'
Skipping line 183262: ';' expected after '"'
Skipping line 210093: ';' expected after '"'
Skipping line 531938: ';' expected after '"'
Skipping line 663237: ';' expected after '"'
Skipping line 824822: ';' expected after '"'
Skipping line 877349: ';' expected after '"'
Skipping line 941679: ';' expected after '"'
Skipping line 1134132: ';' expected after '"'
Skipping line 1145176: ';' expected after '"'
Skipping line 1145177: ';' expected after '"'
Skipping line 1148551: ';' expected after '"'
Skipping line 815: ';' expected after '"'
Skipping line 1679: ';' expected after '"'
Skipping line 2531: ';' expected after '"'
Skipping line 2640: ';' expected after '"'
Skipping line 3236: ';' expected after '"'
Skipping line 3256: ';' expected after '"'
Skipping line 4906: ';' expected after '"'
Skipping line 6452: ';' expected after '"'
Skipping line 8349: ';' ex

Skipping line 68134: ';' expected after '"'
Skipping line 69712: ';' expected after '"'
Skipping line 69920: ';' expected after '"'
Skipping line 70649: ';' expected after '"'
Skipping line 70764: ';' expected after '"'
Skipping line 70826: ';' expected after '"'
Skipping line 71125: ';' expected after '"'
Skipping line 71531: ';' expected after '"'
Skipping line 71615: ';' expected after '"'
Skipping line 71769: ';' expected after '"'
Skipping line 72335: ';' expected after '"'
Skipping line 72372: ';' expected after '"'
Skipping line 72572: ';' expected after '"'
Skipping line 72929: ';' expected after '"'
Skipping line 72930: ';' expected after '"'
Skipping line 72931: ';' expected after '"'
Skipping line 73156: ';' expected after '"'
Skipping line 73162: ';' expected after '"'
Skipping line 73199: ';' expected after '"'
Skipping line 73270: ';' expected after '"'
Skipping line 73849: ';' expected after '"'
Skipping line 73880: ';' expected after '"'
Skipping line 74291: ';' expecte

Skipping line 133250: ';' expected after '"'
Skipping line 133252: ';' expected after '"'
Skipping line 133254: ';' expected after '"'
Skipping line 133362: ';' expected after '"'
Skipping line 133480: ';' expected after '"'
Skipping line 133523: ';' expected after '"'
Skipping line 133625: ';' expected after '"'
Skipping line 133996: ';' expected after '"'
Skipping line 133998: ';' expected after '"'
Skipping line 134001: ';' expected after '"'
Skipping line 134004: ';' expected after '"'
Skipping line 134006: ';' expected after '"'
Skipping line 134007: ';' expected after '"'
Skipping line 135052: ';' expected after '"'
Skipping line 135121: ';' expected after '"'
Skipping line 136649: ';' expected after '"'
Skipping line 136776: ';' expected after '"'
Skipping line 137577: ';' expected after '"'
Skipping line 137613: ';' expected after '"'
Skipping line 137936: ';' expected after '"'
Skipping line 137939: ';' expected after '"'
Skipping line 137965: ';' expected after '"'
Skipping l

In [6]:
#preprocessing
df_rating = df_rating[pd.to_numeric(df_rating['ISBN'], errors='coerce').notnull()]
count_user = df_rating['User-ID'].value_counts()
df_rating_filtered = df_rating[df_rating['User-ID'].isin(count_user[count_user > 20].index)]
count_ISBN = df_rating['ISBN'].value_counts()
df_rating_filtered = df_rating_filtered[df_rating_filtered['ISBN'].isin(count_ISBN[count_ISBN > 49].index)]
df_rating_grp = df_rating_filtered.pivot(index='ISBN', columns='User-ID', values='Book-Rating').fillna(0)
df_rating_grp.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company


In [15]:
#create d6tflow work flow
class TaskGetData(d6tflow.tasks.TaskPickle):
    def run(self):
        data = pd.read_csv("~/Downloads/Book reviews/BX-Book-Ratings.csv", sep=";", quotechar='"', engine='python', error_bad_lines=False)
        self.save(data)
@d6tflow.requires(TaskGetData) # define dependency
class TaskPreprocess(d6tflow.tasks.TaskCachePandas):  # save data in memory
    do_preprocess = luigi.BoolParameter(default=True)

    def run(self):
        data = self.input().load()
        if self.do_preprocess:
            data = data[pd.to_numeric(data['ISBN'], errors='coerce').notnull()]
            #filter out users with fewer than 20 reviews
            count_user = data['User-ID'].value_counts()
            data_filtered = data[data['User-ID'].isin(count_user[count_user > 20].index)]
            #filter out books with fewer than 50 reviews
            count_ISBN = data['ISBN'].value_counts()
            data_filtered = data_filtered[data_filtered['ISBN'].isin(count_ISBN[count_ISBN > 49].index)]
            data_filtered = data_filtered.pivot(index='ISBN', columns='User-ID', values='Book-Rating').fillna(0)
        self.save(data_filtered)
@d6tflow.requires(TaskPreprocess) # automatically pass parameters upstream
class TaskTrain(d6tflow.tasks.TaskPickle): # save output as pickle
    model = luigi.Parameter(default='knn') # parameter for model selection
    def run(self):
        data = self.input().load()
        if self.model=='svd':
            model = TruncatedSVD()
        elif self.model=='knn':
            model = NearestNeighbors(metric = 'cosine', algorithm = 'brute', n_neighbors=21)
        else:
            raise ValueError('invalid model selection')
        model.fit(data)
        self.save(model)

ISBN
 0330299891    3.0
 0375404120    1.5
 0586045007    0.0
 9022906116    3.5
 9032803328    0.0
Name: Book-Rating, dtype: float64

In [19]:
d6tflow.run(TaskTrain())

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Book-Rating
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,0.000000
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,4.928571
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,5.000000
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,4.272727
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,0.000000
5,0399135782,The Kitchen God's Wife,Amy Tan,1991,Putnam Pub Group,4.212121
6,0425176428,What If?: The World's Foremost Military Histor...,Robert Cowley,2000,Berkley Publishing Group,1.600000
7,0671870432,PLEADING GUILTY,Scott Turow,1993,Audioworks,2.666667
8,0679425608,Under the Black Flag: The Romance and the Real...,David Cordingly,1996,Random House,0.000000
9,074322678X,Where You'll Find Me: And Other Stories,Ann Beattie,2002,Scribner,0.000000


In [1]:
def make_recommendations(isbn):
    """
    make top 20 book recommendations
    Parameters
    ----------
    isbn: str
    """
    #load the data to run knn
    data = TaskPreprocess().output().load()
    model = TaskTrain().output().load()
    # get recommendations
    print('Recommendations for {}:'.format(df_book[df_book['ISBN'] == isbn]['Book-Title'].values[0]))
    distances, indices = model.kneighbors(data[data.index == isbn], n_neighbors = 21)
    for i in range(1, len(distances.flatten())):
        try:
            print('{0}: {1}, with distance of {2}:'.format(i, df_book[df_book['ISBN'] == data.iloc[indices.flatten()[i]].name]['Book-Title'].values[0], distances.flatten()[i]))
        except:
            pass
make_recommendations('0060168013')

NameError: name 'df_book_rec' is not defined

In [ ]:
TaskGetData().invalidate()
# reset all downstream tasks
d6tflow.invalidate_downstream(TaskGetData(), TaskTrain())
# reset all upstream tasks
d6tflow.invalidate_upstream(TaskTrain())


In [ ]:
data.loc['0060168013']